In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as st
import pingouin as pg

In [77]:
data = pd.read_csv('/home/atrides/Desktop/R/statistics_with_Python/08_LogisticRegression/Data_Files/eel.dat', sep='\t')
print(data.head())

       Cured  Intervention  Duration
0  Not Cured  No Treatment         7
1  Not Cured  No Treatment         7
2  Not Cured  No Treatment         6
3      Cured  No Treatment         8
4      Cured  Intervention         7


In [78]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Cured         113 non-null    object
 1   Intervention  113 non-null    object
 2   Duration      113 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 2.8+ KB
None


In [79]:
print(data['Cured'].unique())

['Not Cured' 'Cured']


In [80]:
print(data['Intervention'].unique())

['No Treatment' 'Intervention']


In [81]:
data['Curedx'] = data['Cured'].replace({'Not Cured':0,'Cured':1 })
data['Interventionx'] = data['Intervention'].replace({'No Treatment':0,'Intervention':1 })

In [82]:
print(data.head())

       Cured  Intervention  Duration  Curedx  Interventionx
0  Not Cured  No Treatment         7       0              0
1  Not Cured  No Treatment         7       0              0
2  Not Cured  No Treatment         6       0              0
3      Cured  No Treatment         8       1              0
4      Cured  Intervention         7       1              1


In [83]:
from statsmodels.tools.tools import add_constant
data=add_constant(data)

In [112]:
m01 = sm.Logit(data['Curedx'] , data[['const', 'Interventionx']]).fit()
print(m01.summary())

Optimization terminated successfully.
         Current function value: 0.637866
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 Curedx   No. Observations:                  113
Model:                          Logit   Df Residuals:                      111
Method:                           MLE   Df Model:                            1
Date:                Wed, 16 Sep 2020   Pseudo R-squ.:                 0.06442
Time:                        00:45:36   Log-Likelihood:                -72.079
converged:                       True   LL-Null:                       -77.042
Covariance Type:            nonrobust   LLR p-value:                  0.001629
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.2877      0.270     -1.065      0.287      -0.817       0.242
Interventionx     1.

In [113]:
print(m01.summary2())

                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.064    
Dependent Variable: Curedx           AIC:              148.1578 
Date:               2020-09-16 00:45 BIC:              153.6126 
No. Observations:   113              Log-Likelihood:   -72.079  
Df Model:           1                LL-Null:          -77.042  
Df Residuals:       111              LLR p-value:      0.0016294
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     5.0000                                      
----------------------------------------------------------------
                   Coef.  Std.Err.    z    P>|z|   [0.025 0.975]
----------------------------------------------------------------
const             -0.2877   0.2700 -1.0654 0.2867 -0.8169 0.2416
Interventionx      1.2287   0.3998  3.0735 0.0021  0.4452 2.0122



##### AIC =      -2 * log_likelihood + 2 * k

In [114]:
AIC = -2*(-72.079)+ 2*2
print(AIC)

148.158


In [115]:
print(m01.llf)           #get Log-Likelihood
print(m01.llr)           #get chi-square
print(m01.llr_pvalue)    #get sig. level of chi-square test
print(m01.prsquared)     #get pseudo-rsquared 

-72.07890183908266
9.926201217275832
0.0016294247428934145
0.06442071144251205


In [116]:
print(-2*(m01.llnull-m01.llf))

9.926201217275832


In [117]:
print(m01.llr)

9.926201217275832


### **chi-square = deviance_null - deviance_new**

In [118]:
chi_square  = (-2*(-77.042))-(-2*(-72.079))          # previous_deviance - present_deviance
print(chi_square)

9.926000000000016


In [119]:
chi_df = 1
print(1 - st.chi2.cdf(chi_square, chi_df))

0.0016296028825595332


### Seeing above p-value we reject null hypothesis that m01 is not better than just chance at predicting outcome.

### **Note: The model chi-square is an analogue of the F-test for the linear regression**

#### see https://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.LogitResults.html

In [120]:
print(m01.aic)

148.15780367816532


In [121]:
print(m01.bic)

153.61257931559


In [122]:
print(m01.bse)

const            0.270031
Interventionx    0.399759
dtype: float64


In [123]:
print(m01.prsquared)

0.06442071144251205


In [124]:
print(m01.resid_dev)

0     -1.057937
1     -1.057937
2     -1.057937
3      1.301766
4      0.811763
         ...   
108    0.811763
109    0.811763
110   -1.057937
111   -1.057937
112    0.811763
Length: 113, dtype: float64


#### chi_df = This creates a value called chidf that is the degrees of freedom for the model  subtracted from the degrees of freedom for the null model. 

In [125]:
chi_df = 112-111
print(chi_df)

1


In [126]:
z_statsistics = 3.074  # from model
deviance_null = -2*m01.llnull

In [127]:
R  = np.sqrt((z_statsistics**2 - 2*chi_df)/deviance_null)
print(R)

0.21987916734928623


#### The three different R_Squared :

In [128]:
deviance_new = -2*m01.llf
deviance_new

144.15780367816532

In [129]:
def R2(deviance_null, deivance_new, n):
    R2_l = (deviance_null-deviance_new)/deviance_null
    R2_cs = 1-np.exp((deviance_new-deviance_null)/n)
    R2_n = R2_cs/(1-np.exp(-(deviance_null/n)))
    print("pseudo R^2 for logistic regression:\n")
    print(f"Hosmer and Lemeshow R^2: {R2_l}\n")
    print(f"Cox and Snell R^2: {R2_cs} \n")
    print(f"Nagelkerke R^2 : {R2_n}\n")

In [130]:
R2(deviance_null, deviance_new, len(data))

pseudo R^2 for logistic regression:

Hosmer and Lemeshow R^2: 0.06442071144251207

Cox and Snell R^2: 0.08409486918276265 

Nagelkerke R^2 : 0.11299197843845508



In [131]:
print(np.exp(m01.conf_int()))

                      0         1
const          0.441785  1.273245
Interventionx  1.560729  7.479589


In [132]:
print(m01.fittedvalues)

0     -0.287682
1     -0.287682
2     -0.287682
3     -0.287682
4      0.940983
         ...   
108    0.940983
109    0.940983
110   -0.287682
111   -0.287682
112    0.940983
Length: 113, dtype: float64


In [133]:
print(1/(1+np.exp(-(m01.fittedvalues))))

0      0.428571
1      0.428571
2      0.428571
3      0.428571
4      0.719298
         ...   
108    0.719298
109    0.719298
110    0.428571
111    0.428571
112    0.719298
Length: 113, dtype: float64


In [134]:
m02 = sm.Logit(data['Curedx'] , data[['const', 'Interventionx','Duration']]).fit()
print(m02.summary())

Optimization terminated successfully.
         Current function value: 0.637858
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 Curedx   No. Observations:                  113
Model:                          Logit   Df Residuals:                      110
Method:                           MLE   Df Model:                            2
Date:                Wed, 16 Sep 2020   Pseudo R-squ.:                 0.06443
Time:                        00:47:24   Log-Likelihood:                -72.078
converged:                       True   LL-Null:                       -77.042
Covariance Type:            nonrobust   LLR p-value:                  0.006984
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.2347      1.221     -0.192      0.848      -2.627       2.158
Interventionx     1.

In [135]:
print(m02.llf*(-2))     # deviance of m02

144.1558201498999


In [136]:
print(m02.resid_dev.head())

0   -1.057199
1   -1.057199
2   -1.060374
3    1.306008
4    0.810711
dtype: float64


In [137]:
model_chi = -2*(m01.llf - m02.llf)     # previous_deviance - present_deviance
print(model_chi)

0.001983528265412815


In [138]:
chi_df = 111 - 110
print(chi_df)

1


In [139]:
print(1 - st.chi2.cdf(model_chi, chi_df))

0.9644765035956879


### m02 is not much of an improvement over model 1 , by seeing above output.

In [145]:
from statsmodels.genmod import families

In [146]:
res = sm.GLM(data['Curedx'] , data[['const', 'Interventionx','Duration']],
          family=families.Binomial()).fit(attach_wls=True, atol=1e-10)
print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 Curedx   No. Observations:                  113
Model:                            GLM   Df Residuals:                      110
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -72.078
Date:                Wed, 16 Sep 2020   Deviance:                       144.16
Time:                        00:50:29   Pearson chi2:                     113.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.2347      1.221     -0.192

In [147]:
infl = res.get_influence(observed=False)

In [148]:
summ_df = infl.summary_frame()
summ_df.sort_values('standard_resid', ascending=False)[:10]

,dfb_const,dfb_Interventionx,dfb_Duration,cooks_d,standard_resid,hat_diag,dffits_internal
40,-0.197143,-0.170641,0.239493,0.028150,1.198593,0.055520,0.290602
80,-0.197143,-0.170641,0.239493,0.028150,1.198593,0.055520,0.290602
3,-0.089777,-0.137947,0.128271,0.013978,1.177701,0.029347,0.204779
8,-0.089777,-0.137947,0.128271,0.013978,1.177701,0.029347,0.204779
36,-0.089777,-0.137947,0.128271,0.013978,1.177701,0.029347,0.204779
12,0.215020,-0.055470,-0.184177,0.019845,1.171349,0.041587,0.243998
86,0.215020,-0.055470,-0.184177,0.019845,1.171349,0.041587,0.243998
38,0.215020,-0.055470,-0.184177,0.019845,1.171349,0.041587,0.243998
75,0.011466,-0.108796,0.023927,0.008438,1.166456,0.018266,0.159108
78,0.011466,-0.108796,0.023927,0.008438,1.166456,0.018266,0.159108


## **Diagnostic statistics**

### You need to look for cases that might be influencing the logistic regression model:


#### i) Look at standardized residuals and check that no more than 5% of cases have absolute values above 2, and that no more than about 1% have absolute values above 2.5. Any case with a value above about 3 could be an outlier.



#### ii)  Calculate the average leverage (the number of predictors plus 1, divided by the sample size) and then look for values greater than twice or three times this average value.



#### iii)  Look for absolute values of DFBeta greater than 1.

#### all cases have DFBetas less than 1, and leverage statistics are very close to the calculated expected value of 0.018.
#### All in all, this means that there are no influential cases having an effect on the model. 
#### The studentized residuals all have values of less than ±2 and so there seems to be very little here to concern us.

# Another Example

In [157]:
pen_df = pd.read_csv('/home/atrides/Desktop/R/statistics_with_Python/08_LogisticRegression/Data_Files/penalty.dat', sep='\t')

In [161]:
print(pen_df.head())

   PSWQ  Anxious  Previous          Scored
0    18       21        56  Scored Penalty
1    17       32        35  Scored Penalty
2    16       34        35  Scored Penalty
3    14       40        15  Scored Penalty
4     5       24        47  Scored Penalty


In [162]:
pen_df['Scored'].unique()

array(['Scored Penalty', 'Missed Penalty'], dtype=object)

In [163]:
pen_df['Scoredx']  = pen_df['Scored'].replace({'Scored Penalty':1, 'Missed Penalty':0})

In [164]:
pen_df = add_constant(pen_df)

In [175]:
p01 = sm.Logit(pen_df['Scoredx'], pen_df[['const', 'PSWQ', 'Previous']]).fit()
print(p01.summary())

Optimization terminated successfully.
         Current function value: 0.324412
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                Scoredx   No. Observations:                   75
Model:                          Logit   Df Residuals:                       72
Method:                           MLE   Df Model:                            2
Date:                Wed, 16 Sep 2020   Pseudo R-squ.:                  0.5305
Time:                        00:56:56   Log-Likelihood:                -24.331
converged:                       True   LL-Null:                       -51.819
Covariance Type:            nonrobust   LLR p-value:                 1.153e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2803      1.670      0.767      0.443      -1.993       4.554
PSWQ          -0.2301      0.

In [176]:
model1_chi_sq = -2*(p01.llnull - p01.llf)   # p01.llr
print(model1_chi_sq)

54.97669239214833


In [177]:
chi_df1 = 2

In [180]:
chisq_prob1 =  print(1 - st.chi2.cdf(model1_chi_sq, chi_df1))   # significant p-value

1.1532996779806126e-12


### the chisquare probability 'chisq_prob1' value is less than 0.05 which tells that this model was quite an improvement over a null model(i.e just chance)

In [183]:
p02 = sm.Logit(pen_df['Scoredx'], pen_df[['const',  'PSWQ', 'Previous', 'Anxious']]).fit()
print(p02.summary())

Optimization terminated successfully.
         Current function value: 0.316104
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                Scoredx   No. Observations:                   75
Model:                          Logit   Df Residuals:                       71
Method:                           MLE   Df Model:                            3
Date:                Wed, 16 Sep 2020   Pseudo R-squ.:                  0.5425
Time:                        00:59:09   Log-Likelihood:                -23.708
converged:                       True   LL-Null:                       -51.819
Covariance Type:            nonrobust   LLR p-value:                 3.765e-12
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.4926     11.802     -0.974      0.330     -34.624      11.638
PSWQ          -0.2514      0.

In [184]:
model2_chi_sq = -2*(p01.llf - p02.llf)   
print(model2_chi_sq)

1.246266751259725


In [185]:
chi_df2 = 1

In [187]:
chisq_prob2 =  print(1 - st.chi2.cdf(model2_chi_sq, 1))   # non-significant p-value , so no improvement 

0.2642667072391809


###  the chisquare probability 'chisq_prob2' value is greater than 0.05 , which tells that this model(i.e p02) was a improvement over p01  , just by chance.

In [188]:
print(p01.aic)
print(p02.aic)

54.66180402298753
55.41553727172781


In [189]:
print(p01.bic)
print(p02.bic)

61.614268363596466
64.68548972587305


In [190]:
copy_df = pen_df.copy()

In [191]:
copy_df.drop(['Scored','Scoredx'], inplace=True, axis=1)

In [192]:
copy_df .head()

,const,PSWQ,Anxious,Previous
0,1.0,18,21,56
1,1.0,17,32,35
2,1.0,16,34,35
3,1.0,14,40,15
4,1.0,5,24,47


In [193]:
cov = p02.cov_params()
corr = cov / p02.bse / p02.bse[:, None]
vif = np.diag(np.linalg.inv(corr.values[1:, 1:]))[[1, 0, 2]]

In [196]:
print(vif)

[35.22707635  1.08976625 35.58192988]


#### https://stats.stackexchange.com/questions/474964/calculating-variance-inflation-factor-for-logistic-regression-using-statsmodels/475233#475233

In [197]:
tolerance = 1/vif
print(tolerance)

[0.02838726 0.91762798 0.02810415]


### from the output of  vif and tolerance , we can deduce that there is a high multicollinearity in our model 

### **Testing for linearity of the logit**

In [198]:
pen_df.head()

,const,PSWQ,Anxious,Previous,Scored,Scoredx
0,1.0,18,21,56,Scored Penalty,1
1,1.0,17,32,35,Scored Penalty,1
2,1.0,16,34,35,Scored Penalty,1
3,1.0,14,40,15,Scored Penalty,1
4,1.0,5,24,47,Scored Penalty,1


In [199]:
pen_df['log_PSWQ'] = np.log(pen_df['PSWQ'])*pen_df['PSWQ']
pen_df['log_Anxious'] = np.log(pen_df['Anxious'])*pen_df['Anxious']
pen_df['log_Previous'] = np.log(pen_df['Previous']+1)*pen_df['Previous']

In [201]:
p03 = sm.Logit(pen_df['Scoredx'], pen_df[['const',  'PSWQ', 'Previous', 'Anxious','log_PSWQ', 'log_Anxious', 'log_Previous']]).fit()
print(p03.summary())

Optimization terminated successfully.
         Current function value: 0.306062
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                Scoredx   No. Observations:                   75
Model:                          Logit   Df Residuals:                       68
Method:                           MLE   Df Model:                            6
Date:                Wed, 16 Sep 2020   Pseudo R-squ.:                  0.5570
Time:                        01:04:21   Log-Likelihood:                -22.955
converged:                       True   LL-Null:                       -51.819
Covariance Type:            nonrobust   LLR p-value:                 1.300e-10
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -3.5721     15.008     -0.238      0.812     -32.988      25.844
PSWQ            -0.4222

## From the summary output , if any interaction term has significance less than 0.05 , it will mean that assumption of linearity has been violated. In our output we can conclude that the assumption of linearity has been met as all interaction term is non-significant